# WEIGTHS AND DIMENTIONS MODEL

In [284]:
# Librerias a usar

import pyodbc
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from pmdarima import auto_arima
from tqdm import tqdm 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import sqlalchemy
from sqlalchemy import create_engine, types
import pyodbc
import smtplib
from email.mime.text import MIMEText
from sqlalchemy import create_engine, Table, Column, MetaData, Integer, String, Float
import os

In [195]:
def send_email_inicio():
    # Configuración del correo
   
    # 🔹 Lista de destinatarios
    recipient_list = ["ecalderon@maxwarehouse.com"]
    
    # Contenido del correo
    subject = "🔔 Alerta: Inicio de generación de predicciones de pesos"
    body = "El modelo de predicción ha empezado a correr."

    # Crear el mensaje
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = ", ".join(recipient_list)

    try:
        # Conectar al servidor SMTP (ejemplo para Gmail)
        server = smtplib.SMTP("smtp.office365.com", 587)  # Usa "smtp.office365.com" para Outlook
        server.starttls()  # Seguridad TLS
        server.login(sender_email, sender_password)  # Autenticación
        server.sendmail(sender_email, recipient_list, msg.as_string())  # Enviar
        server.quit()
        print("✅ Correo enviado correctamente")
    except Exception as e:
        print(f"⚠ Error al enviar el correo: {e}")

# Llamar a la función al final del proceso
send_email_inicio()

✅ Correo enviado correctamente


## Incorporar reporte semanal de endicia

In [285]:
# --- Configuración ---

# Ruta del archivo CSV
carpeta = r'C:\Users\ecalderon\OneDrive - BITS - Max Warehouse\Shipping\Weights Process\Reporte Semanal Endicia'

# Listar todos los archivos .xlsx de la carpeta
excel_files = [f for f in os.listdir(carpeta) if f.endswith('.xlsx')]

# Verificar que haya archivos .xlsx
if not excel_files:
    raise FileNotFoundError("No se encontraron archivos Excel (.xlsx) en la carpeta.")

# Seleccionar el último archivo modificado
excel_files = sorted(excel_files, key=lambda f: os.path.getmtime(os.path.join(carpeta, f)))
ultimo_excel = excel_files[-1]

# Ruta completa al archivo
ruta_archivo = os.path.join(carpeta, ultimo_excel)

# Leer el archivo Excel
df = pd.read_excel(ruta_archivo)

print(f"Archivo leído: {ultimo_excel}")

# Datos de conexión a SQL Server
server = '10.12.200.59' #ip del DB es 10.11.100.103
database = 'Maxwarehouse'
table_name = 'endicia_semanal'

# Crear conexión usando SQLAlchemy y Trusted Connection
conexion_str = f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server;trusted_connection=yes"
engine = create_engine(conexion_str)

Archivo leído: Account Transactional Activity.xlsx


In [286]:
# sqlalchemy + pyodbc
connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
engine = create_engine(connection_string)

# --- 2. Abrir conexión y cargar los datos ---
with engine.connect() as connection:
    # Si la tabla NO existe, la crea. Si existe, agrega (append).
    df.to_sql(table_name, con=connection, if_exists='append', index=False)

print("Datos cargados exitosamente a SQL Server 🎯")

Datos cargados exitosamente a SQL Server 🎯


## QUERY PESOS Y DIMENSIONES

In [198]:
server = '' 
database = ''

# Crear el engine usando la cadena de conexión para SQL Server con pyodbc
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Ejecutar consulta 

query = """

WITH BOXFECHAS AS (
	Select TRACKING_NUMBER,OrderDate, case when len(DIMENSIONS)>=5 then CAST(SUBSTRING(DIMENSIONS, 1, CHARINDEX('x', DIMENSIONS) - 1) AS float) else 0.00 end DimHeightBOX,
			case when len(DIMENSIONS)>=5 then CAST(SUBSTRING(DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1, 
								CHARINDEX('x', DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1) - CHARINDEX('x', DIMENSIONS) - 1) AS float)  else 0.00 end DimLengthBOX, 
			case when len(DIMENSIONS)>=5 then CAST(RIGHT(DIMENSIONS, LEN(DIMENSIONS) - CHARINDEX('x', DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1)) AS float)  else 0.00 end  DimWidthBOX, 
			CCN_ORDER_NUMBER
			FROM 
				BOX 
			WHERE ISDATE(OrderDate) = 1),

endicia_semana_mes AS (
		SELECT DISTINCT  
		[Postmark_Date] AS TranDate, 
		[Transaction_Date (GMT)] AS InvDate, 
		REPLACE([Tracking Number], '''', '') AS trackingnum, 
		[Weight_(lb)], 
		[Package Height], 
		[Package Length], 
		[Package Width]
	FROM Endiciatransacprovision
	WHERE CAST([Transaction_Date (GMT)] AS date) >= DATEADD(MONTH, -4, GETDATE())
	UNION ALL
	SELECT DISTINCT  
		[Postmark Date] AS TranDate, 
		[Transaction Date (GMT)] AS InvDate, 
		REPLACE([Tracking Number], '''', '') AS trackingnum, 
		[Weight (lb)], 
		[Package Height], 
		[Package Length], 
		[Package Width]
	FROM endicia_semanal
	WHERE CAST([Transaction Date (GMT)] AS date) >= DATEADD(MONTH, -2, GETDATE())
),

Factura3meses AS (
	SELECT DISTINCT
        cast(InvoiceDate as date) TranDate, cast(InvoiceDate as date) InvDate, GroundTrackingIDPrefix+ExpressorGroundTrackingID trackingnum,
        MAX(ActualWeightAmount) AS max_weight, 
		CASE WHEN DimHeight !='' THEN cast(DimHeight as float) ELSE  DimHeightBOX END DimHeight,
		CASE WHEN DimLength !='' THEN cast(DimLength as float) ELSE  DimLengthBOX END DimLength, 
		CASE WHEN DimWidth !='' THEN cast(DimWidth as float) ELSE  DimWidthBOX END DimWidth,
		max(CCN_ORDER_NUMBER) CCN_ORDER_NUMBER
    FROM 
        FEDEXHISTORICO
	LEFT JOIN 
			BOXFECHAS ON GroundTrackingIDPrefix+ExpressorGroundTrackingID = TRACKING_NUMBER
    WHERE 
        cast(InvoiceDate as date) >= DATEADD(MONTH, -3, GETDATE()) 
    GROUP BY 
        cast(InvoiceDate as date), cast(InvoiceDate as date), GroundTrackingIDPrefix+ExpressorGroundTrackingID,
		CASE WHEN DimHeight !='' THEN cast(DimHeight as float) ELSE  DimHeightBOX END ,
		CASE WHEN DimLength !='' THEN cast(DimLength as float) ELSE  DimLengthBOX END , 
		CASE WHEN DimWidth !='' THEN cast(DimWidth as float) ELSE  DimWidthBOX END 
    UNION ALL
    SELECT 
        cast(TransactionDate as date) TranDate, cast(InvoiceDate as date) InvDate,TrackingNumber,
        max(case when BilledWeight != 0 then BilledWeight else EnteredWeight end) as max_weight,
		CASE WHEN PackageDimensions !='' THEN 
			CAST(SUBSTRING(PackageDimensions, 1, CHARINDEX('x', PackageDimensions) - 1) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(SUBSTRING(DetailKeyedDim, 1, CHARINDEX('x', DetailKeyedDim) - 1) AS float) else DimHeightBOX end END AS Valor1,

		CASE WHEN PackageDimensions !='' THEN 
			CAST(SUBSTRING(PackageDimensions, CHARINDEX('x', PackageDimensions) + 1, 
                        CHARINDEX('x', PackageDimensions, CHARINDEX('x', PackageDimensions) + 1) - CHARINDEX('x', PackageDimensions) - 1) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(SUBSTRING(DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1, 
                        CHARINDEX('x', DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1) - CHARINDEX('x', DetailKeyedDim) - 1) AS float)  else DimLengthBOX end END  AS Valor2,

		CASE WHEN PackageDimensions !='' THEN 
			CAST(RIGHT(PackageDimensions, LEN(PackageDimensions) - CHARINDEX('x', PackageDimensions, CHARINDEX('x', PackageDimensions) + 1)) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(RIGHT(DetailKeyedDim, LEN(DetailKeyedDim) - CHARINDEX('x', DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1)) AS float)  else DimWidthBOX end END  AS Valor3,
			max(CCN_ORDER_NUMBER)CCN_ORDER_NUMBER
    FROM 
        UPSNEWFORMAT
	LEFT JOIN 
			BOXFECHAS ON TrackingNumber = TRACKING_NUMBER
    WHERE 
        cast(InvoiceDate as date) >= DATEADD(MONTH, -3, GETDATE())
    GROUP BY 
        cast(TransactionDate as date), cast(InvoiceDate as date), TrackingNumber,
		CASE WHEN PackageDimensions !='' THEN 
			CAST(SUBSTRING(PackageDimensions, 1, CHARINDEX('x', PackageDimensions) - 1) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(SUBSTRING(DetailKeyedDim, 1, CHARINDEX('x', DetailKeyedDim) - 1) AS float) else DimHeightBOX end END ,

		CASE WHEN PackageDimensions !='' THEN 
			CAST(SUBSTRING(PackageDimensions, CHARINDEX('x', PackageDimensions) + 1, 
                        CHARINDEX('x', PackageDimensions, CHARINDEX('x', PackageDimensions) + 1) - CHARINDEX('x', PackageDimensions) - 1) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(SUBSTRING(DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1, 
                        CHARINDEX('x', DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1) - CHARINDEX('x', DetailKeyedDim) - 1) AS float)  else DimLengthBOX end END ,

		CASE WHEN PackageDimensions !='' THEN 
			CAST(RIGHT(PackageDimensions, LEN(PackageDimensions) - CHARINDEX('x', PackageDimensions, CHARINDEX('x', PackageDimensions) + 1)) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(RIGHT(DetailKeyedDim, LEN(DetailKeyedDim) - CHARINDEX('x', DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1)) AS float)  else DimWidthBOX end END 

	UNION ALL
	SELECT DISTINCT  TranDate, InvDate, trackingnum, 
	[Weight_(lb)], case when [Package Height] is not null then [Package Height] else DimHeightBOX END , 
	case when [Package Length] is not null then [Package Length] else DimHeightBOX END, 
	case when [Package Width] is not null then [Package Width] else DimHeightBOX END , CCN_ORDER_NUMBER 
	FROM 
		endicia_semana_mes
	LEFT JOIN BOXFECHAS ON TRACKING_NUMBER = trackingnum
	WHERE trackingnum IS NOT NULL
	and cast(InvDate as date) >= DATEADD(MONTH, -4, GETDATE()) --se colocan 4 meses porque las dims de USPS vienen mensualmente
),

DWVentasUnicas as (
	Select 
		SalesOrderNumber, Sku
	from DWShippingInfo as dws
	where cast(SalesOrderDate as date ) between dateadd(MONTH,-4, cast(GETDATE() as date)) and cast(GETDATE() as date)
	group by SalesOrderNumber, Sku
),

CantidadSkusxVenta as (
	select
		SalesOrderNumber, count(Sku) CantidadSkus
	from DWVentasUnicas dw 
	group by SalesOrderNumber
	having count(Sku) = 1
),

PesosfacturaVentas as (
	--Lista los trackings con su peso y le coloca la moda y las frecuencia con que ese peso fue facturado
	select  
		TranDate, InvDate, dw.trackingnum, max_weight, DimHeight, DimLength, DimWidth, mw.Sku, mw.UomCode, mw.UomQuantity
	FROM Factura3meses dw
	INNER JOIN DWShippingInfo  mw
		ON dw.trackingnum = mw.trackingnum
	WHERE dw.trackingnum !=''
	and SalesOrderNumber in (select SalesOrderNumber from CantidadSkusxVenta)
	and cast(UomQuantity as float)/cast(Quantity as float)= 1
),

RankedData AS (
    SELECT 
        TranDate, 
        InvDate, 
        trackingnum, 
        Sku, 
        UomCode, 
        UomQuantity,
        max_weight, 
        DimHeight, 
        DimLength, 
        DimWidth,
        RANK() OVER (
            PARTITION BY TranDate, trackingnum, Sku, UomCode 
            ORDER BY InvDate DESC
        ) AS Ranking
    FROM PesosfacturaVentas
),

Max_Rank as (SELECT 
    TranDate, 
    InvDate, 
    trackingnum, 
    Sku, 
    UomCode, 
    UomQuantity,
    MAX(max_weight) AS Weight, 
    MAX(DimHeight) AS Height, 
    MAX(DimLength) AS Length, 
    MAX(DimWidth) AS Width,
    Ranking
FROM RankedData
GROUP BY TranDate, InvDate, trackingnum, Sku, UomCode, UomQuantity, Ranking
),

EJD_DIMS as (SELECT aa.*, 
       bb.[EJD Weight], 
       bb.[EJD Height], 
       bb.[EJD Length], 
       bb.[EJD Width],
       bb.[BaseUnit]
FROM Max_Rank aa
LEFT JOIN (
    SELECT DISTINCT [EVP SKU], [EVP UOM CODE], [EVP UOM QTY], [EJD Weight], [EJD Height], [EJD Length], [EJD Width], [BaseUnit]
    FROM [stg-MaxWarehouse].[EvpCsv].[EVPAlllSKUsDimNWeight]
) bb
ON aa.Sku COLLATE SQL_Latin1_General_CP1_CI_AS = bb.[EVP SKU] COLLATE SQL_Latin1_General_CP1_CI_AS
and aa.UomCode COLLATE SQL_Latin1_General_CP1_CI_AS= bb.[EVP UOM CODE] COLLATE SQL_Latin1_General_CP1_CI_AS
),

Ventas as (
SELECT * FROM [DWH-MaxWarehouse].[Fact].[Sales]
WHERE cast(SalesOrderDateTime as date) >= DATEADD(MONTH, -6, GETDATE())  
)

select distinct aa.*, bb.SalesStatusEVP, bb.TotalSales, bb.TotalCost from EJD_DIMS aa
LEFT JOIN  Ventas bb
ON aa.trackingnum COLLATE SQL_Latin1_General_CP1_CI_AS = bb.TrackingNumber COLLATE SQL_Latin1_General_CP1_CI_AS
and aa.Sku COLLATE SQL_Latin1_General_CP1_CI_AS = bb.ParentSku COLLATE SQL_Latin1_General_CP1_CI_AS

"""

df = pd.read_sql(query, con=engine)

In [199]:
df = df.rename(columns={
    "EJD Weight": "EJD_Weight",
    "EJD Height": "EJD_Height",
    "EJD Length": "EJD_Length",
    "EJD Width": "EJD_Width"
})

## Limpiar UOMCODE

In [200]:
def normalize_uom_code(df, column='UomCode'):
    """
    Normaliza la columna UomCode:
    - Convierte a mayúsculas
    - Elimina espacios innecesarios
    - Reemplaza valores según las reglas definidas
    - Convierte formatos como "CASE_X" → "CS/X" y "PACK OF X" → "PK/X"
    
    Parámetros:
        df (pd.DataFrame): DataFrame con la columna UomCode
        column (str): Nombre de la columna a normalizar
    
    Retorna:
        pd.DataFrame: DataFrame con una nueva columna 'UomCode_N' normalizada
    """
    df = df.copy()
    
    # Convertir a mayúsculas y eliminar espacios innecesarios
    df['UomCode_N'] = df[column].str.strip().str.upper()

    # Reemplazar "CASE_X" → "CS/X" antes de otros reemplazos
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^CASE[_\s]?(\d+)', r'CS/\1', regex=True)

    # Reemplazar caracteres no uniformes y homogenizar
    replacements = {
        r'\bEACH\b': 'EA',  # EACH → EA
        r'\bEA\b': 'EA',    # Variaciones de EA → EA
        r'\bCASE\b': 'CS',  # CASE → CS
        r'\bPACK\b': 'PK',  # PACK → PK
        r'_' : '/'         # Reemplazar "_" por "/"
    }
    
    df['UomCode_N'] = df['UomCode_N'].replace(replacements, regex=True)

    # Convertir "PACK/X" a "PK/X"
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^PACK/', 'PK/', regex=True)
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^PA/', 'PK/', regex=True)

    # Convertir "CS OF X" → "CS_X" y "PK OF X" → "PK_X"
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^CS OF (\d+)', r'CS/\1', regex=True)
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^PK OF (\d+)', r'PK/\1', regex=True)

    # Reemplazar '/' por '_'
    df['UomCode_N'] = df['UomCode_N'].str.replace('/', '_', regex=False)

    # Reemplazar "ACE" por "EA" y quitar el resto de la cadena
    df['UomCode_N'] = df['UomCode_N'].str.replace(r'^ACE.*', 'EA', regex=True)

    return df

In [201]:
df = normalize_uom_code(df)

## Tomar el valor más reciente para pesos y dimensiones por transaccion

In [202]:
df2 = df.sort_values(by=['TranDate', 'trackingnum', 'Sku', 'UomCode_N', 'InvDate'], ascending=[True, True, True, True, False])

cols_to_fill = ['Weight', 'Height', 'Length', 'Width']
df2[cols_to_fill] = df2[cols_to_fill].replace(0, pd.NA)

df2[cols_to_fill] = df2.groupby(['TranDate', 'trackingnum', 'Sku', 'UomCode_N'])[cols_to_fill].transform(lambda x: x.bfill())

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\179128486.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2[cols_to_fill] = df2.groupby(['TranDate', 'trackingnum', 'Sku', 'UomCode_N'])[cols_to_fill].transform(lambda x: x.bfill())


## Limpieza

In [203]:
# Ordenar por TranDate, trackingnum, Sku, UomCode_N, y InvDate DESC
df2 = df2.sort_values(by=['TranDate', 'trackingnum', 'Sku', 'UomCode_N', 'InvDate'], ascending=[True, True, True, True, False])

# Asignar ranking dentro de cada grupo según InvDate DESC (el primero será Rank 1)
df2['Rank'] = df2.groupby(['TranDate', 'trackingnum', 'Sku', 'UomCode_N']).cumcount() + 1

# Filtrar solo los de Ranking = 1
df_final = df2[df2['Rank'] == 1]

In [204]:
# Eliminar nulos

df_final = df_final.dropna(subset=['TranDate', 'InvDate', 'trackingnum', 'Sku', 'UomCode_N', 'Weight', 'Weight', 'Length', 'Width'])
df_final.fillna(0, inplace=True)

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\2390383199.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final.fillna(0, inplace=True)


In [205]:
df_final

,TranDate,InvDate,trackingnum,Sku,UomCode,UomQuantity,Weight,Height,Length,Width,...,EJD_Weight,EJD_Height,EJD_Length,EJD_Width,BaseUnit,SalesStatusEVP,TotalSales,TotalCost,UomCode_N,Rank
60,2025-02-10,2025-02-15,1Z53E7100300323119,ACE-72159,PACK_24,24.0,7.0,18.0,16.0,4.0,...,4.70,3.50,12.2,6.00,False,0,0.0,0.0,PK_24,1
120,2025-02-10,2025-02-15,1Z53E7100300323128,ACE-6338933,EA,1.0,2.0,10.0,6.0,6.0,...,0.19,8.55,2.15,5.55,True,0,0.0,0.0,EA,1
121,2025-02-10,2025-02-15,1Z53E7100300323146,ACE-1365345,PACK_4,4.0,7.0,10.0,6.0,6.0,...,1.48,9.40,2.65,2.70,False,0,0.0,0.0,PK_4,1
390,2025-02-10,2025-02-15,1Z53E7100350340882,ACE-50996,EA,1.0,20.0,30.0,20.0,10.0,...,11.3,25.5,12.3,15.5,True,0,0.0,0.0,EA,1
330,2025-02-10,2025-02-15,1Z53E7100350340891,DHD-270864,PACK_1,1.0,25.0,18.0,16.0,10.0,...,23.8,12.6,10.1,8.15,True,0,0.0,0.0,PK_1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132068,2025-04-04,2025-04-04,705542524859,ACE-54060,PACK_1,1.0,80.0,10.0,68.0,10.0,...,79.5,9.50,56.5,9.50,True,0,0.0,0.0,PK_1,1
137715,2025-04-04,2025-04-04,705542525053,EJD-8093917,EA,1.0,149.0,20.0,39.0,25.0,...,149.,20.2,25.0,39.0,True,0,0.0,0.0,EA,1
132811,2025-04-04,2025-04-04,705542525064,EJD-8093915,EA,1.0,99.0,19.0,32.0,22.0,...,99.2,18.7,32.0,22.0,True,0,0.0,0.0,EA,1
133812,2025-04-04,2025-04-04,705553207822,EJD-8093917,EA,1.0,149.0,20.0,39.0,25.0,...,149.,20.2,25.0,39.0,True,0,0.0,0.0,EA,1


In [206]:
#Convertir tipo de dato

df_final['Weight'] = df_final['Weight'].astype(str).astype(float)
df_final['Height'] = df_final['Height'].astype(str).astype(float)
df_final['Length'] = df_final['Length'].astype(str).astype(float)
df_final['Width'] = df_final['Width'].astype(str).astype(float)
df_final['EJD_Weight'] = df_final['EJD_Weight'].astype(str).astype(float)
df_final['EJD_Length'] = df_final['EJD_Length'].astype(str).astype(float)
df_final['EJD_Height'] = df_final['EJD_Height'].astype(str).astype(float)
df_final['EJD_Width'] = df_final['EJD_Width'].astype(str).astype(float)
df_final['TranDate'] = pd.to_datetime(df_final['TranDate'], format='%d-%m-%Y')
df_final['InvDate'] = pd.to_datetime(df_final['InvDate'], format='%d-%m-%Y')
df_final['UomQuantity'] = df_final['UomQuantity'].astype(str).astype(float).round(1)

In [207]:
# Ordenar Dimensiones de mayor a menor
df_sorted = df_final.copy()
df_sorted[['Height', 'Length', 'Width']] = df_final[['Height', 'Length', 'Width']].apply(
    lambda row: sorted(row, reverse=True), axis=1, result_type='expand')

df_sorted[['EJD_Height', 'EJD_Length', 'EJD_Width']] = df_final[['EJD_Height', 'EJD_Length', 'EJD_Width']].apply(
    lambda row: sorted(row, reverse=True), axis=1, result_type='expand')

## Outliers

In [208]:
# Función para reemplazar outliers por NaN por Sku usando IQR
def replace_outliers_with_nan(group):
    Q1 = group[['Weight', 'Height', 'Length', 'Width']].quantile(0.25)
    Q3 = group[['Weight', 'Height', 'Length', 'Width']].quantile(0.75)
    IQR = Q3 - Q1
    factor = 0.8

    # Detectar outliers
    is_outlier = ((group[['Weight', 'Height', 'Length', 'Width']] < (Q1 - factor * IQR)) |
                  (group[['Weight', 'Height', 'Length', 'Width']] > (Q3 + factor * IQR)))

    # Reemplazar outliers por NaN
    group[['Weight', 'Height', 'Length', 'Width']] = group[['Weight', 'Height', 'Length', 'Width']].mask(is_outlier, np.nan)

    return group

# Aplicar la función por cada Sku
df_cleaned = df_sorted.groupby(['Sku', 'UomCode_N']).apply(replace_outliers_with_nan)

# Reiniciar el índice si deseas (opcional)
df_cleaned = df_cleaned.reset_index(drop=True)

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\1995558355.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_cleaned = df_sorted.groupby(['Sku', 'UomCode_N']).apply(replace_outliers_with_nan)


In [209]:
# Rellenar Vacios de outliers con la mediana
df_cleaned[['Weight', 'Height', 'Length', 'Width']] = df_cleaned.groupby(['Sku', 'UomCode_N'])[['Weight', 'Height', 'Length', 'Width']].transform(lambda x: x.fillna(x.median()))

## MODELO PREDICTIVO

### Random Forest

In [210]:
df = df_cleaned.copy()

In [211]:
# Codificar Sku como numérico
label_encoder = LabelEncoder()
le = LabelEncoder()
df['Sku_encoded'] = le.fit_transform(df['Sku'])

# Codificar UomCode_N como numérico
le_uomcode = LabelEncoder()
df['UomCode_N_encoded'] = le_uomcode.fit_transform(df['UomCode_N'])

In [212]:
# Umbral de registros
umbral_min_registros = 5

# Separar los SKUs con pocos y muchos registros, considerando también UomCode_N
sku_uomcode_counts = df.groupby(['Sku', 'UomCode_N']).size()

# Filtrar SKUs con muchos registros (más de 5)
skus_muchos = sku_uomcode_counts[sku_uomcode_counts > umbral_min_registros].index
skus_pocos = sku_uomcode_counts[sku_uomcode_counts <= umbral_min_registros].index

# Separar en dos bases considerando tanto Sku como UomCode_N
df_muchos = df[df.set_index(['Sku', 'UomCode_N']).index.isin(skus_muchos)]
df_pocos = df[df.set_index(['Sku', 'UomCode_N']).index.isin(skus_pocos)]

In [213]:
# Identificar los SKUs con al menos un registro con peso menor a 1
skus_with_weight_below_1 = df_muchos[df_muchos['Weight'] < 1]['Sku'].unique()

# Identificar los SKUs con al menos un registro con peso menor a 1, considerando uomcode
skus_uomcode_with_weight_below_1 = df_muchos[df_muchos['Weight'] < 1] \
    .groupby(['Sku', 'UomCode_N']) \
    .size() \
    .reset_index(name='count')

In [214]:
# Filtrar los datos en dos grupos
# Grupo 1: SKUs con al menos un registro con peso menor a 1
df_group_1 = df_muchos[df_muchos['Sku'].isin(skus_with_weight_below_1)]

# Grupo 2: El resto de los SKUs (peso mayor o igual a 1 en todos los registros)
df_group_2 = df_muchos[~df_muchos['Sku'].isin(skus_with_weight_below_1)]

# considerando uomcode
# Crear una lista de combinaciones Sku y UomCode_N que tienen peso menor a 1
skus_uomcode_below_1 = skus_uomcode_with_weight_below_1[['Sku', 'UomCode_N']]

# Grupo 1: SKUs con al menos un registro con peso menor a 1
df_group_1 = df_muchos[df_muchos.set_index(['Sku', 'UomCode_N']).index.isin(skus_uomcode_below_1.set_index(['Sku', 'UomCode_N']).index)]

# Grupo 2: El resto de los SKUs (peso mayor o igual a 1 en todos los registros)
df_group_2 = df_muchos[~df_muchos.set_index(['Sku', 'UomCode_N']).index.isin(skus_uomcode_below_1.set_index(['Sku', 'UomCode_N']).index)]

## Modelo Por Grupo Prediccion Semana

### Modelo para pocos registros (mediana)

In [215]:
def predict_with_median_or_model(group_df):
    # Crear un DataFrame vacío para las predicciones
    predictions = group_df.copy()
    predictions = predictions.iloc[:,3:]
    
    # Calcular la mediana para cada SKU (por SKU)
    median_values = group_df.groupby(['Sku_encoded', 'UomCode_N_encoded'])[['Weight', 'Height', 'Length', 'Width']].transform('median')
    
    # Asignar la mediana a las predicciones para los SKUs con menos de 5 registros
    predictions[['Weight', 'Height', 'Length', 'Width']] = median_values

    # Eliminar filas duplicadas
    predictions = predictions.drop_duplicates()
    
    return predictions

In [216]:
predict_pocos = predict_with_median_or_model(df_pocos)[["Sku", 'UomCode_N',"Weight", "Height", "Length", "Width"]].drop_duplicates()

### Modelo para muchos registros (random forest)

In [217]:
# Función para entrenar modelos por grupo sin hacer split y con validación cruzada

def train_models(group_df):

    # Eliminar duplicados 
    group_df = group_df.drop_duplicates(subset=['Sku_encoded', 'UomCode_N_encoded'])
    
    X = group_df[['Sku_encoded', 'UomCode_N_encoded']]
    y = group_df[['Weight', 'Height', 'Length', 'Width']]

    models = {}
    for target in ['Weight', 'Height', 'Length', 'Width']:
        model = RandomForestRegressor(
            n_estimators=150,  # Más árboles para estabilidad
            max_depth=10,  # Limitar profundidad para evitar sobreajuste
            min_samples_split=5,  # Requiere al menos 5 muestras para dividir un nodo
            random_state=42,
            n_jobs=-1)  # Usa todos los núcleos del procesador
        
        # Evaluación con Cross-Validation (3 folds)
        scores = cross_val_score(model, X, y[target], cv=2, scoring='neg_mean_absolute_error')
        mae = np.mean(np.abs(scores))  # Convertimos a MAE positivo
        
        print(f'MAE {target} (cross-validation): {mae:.4f}')
        
        # Entrenar con todos los datos después de la validación
        model.fit(X, y[target])
        models[target] = model

    return models

# Entrenar modelos por grupo
print("Entrenando modelos para grupo 1 (peso < 1)")
rf_models_1 = train_models(df_group_1)

print("Entrenando modelos para grupo 2 (peso >= 1)")
rf_models_2 = train_models(df_group_2)

Entrenando modelos para grupo 1 (peso < 1)
MAE Weight (cross-validation): 0.4294
MAE Height (cross-validation): 3.4217
MAE Length (cross-validation): 2.5496
MAE Width (cross-validation): 2.3087
Entrenando modelos para grupo 2 (peso >= 1)
MAE Weight (cross-validation): 11.8511
MAE Height (cross-validation): 8.5386
MAE Length (cross-validation): 4.6150
MAE Width (cross-validation): 3.5434


In [218]:
# Función para predecir una lista de SKUs
def predict_skus(skus, uomcodes, models, le, le_uom):
    skus_encoded = [le.transform([sku])[0] for sku in skus if sku in le.classes_]
    uomcodes_encoded = [le_uom.transform([uomcode])[0] for uomcode in uomcodes if uomcode in le_uom.classes_]

    if not skus_encoded or not uomcodes_encoded:
        print("Ningún SKU o UOMCode válido para predecir.")
        return pd.DataFrame()

    # Crear el DataFrame con los SKUs y sus UOMCodes codificados
    df_pred = pd.DataFrame({'Sku': skus, 
                            'UomCode_N': uomcodes, 
                            'Sku_encoded': skus_encoded, 
                            'UomCode_N_encoded': uomcodes_encoded})

    # Realizar las predicciones con los modelos
    predictions = {target: models[target].predict(df_pred[['Sku_encoded', 'UomCode_N_encoded']]) 
                   for target in ['Weight', 'Height', 'Length', 'Width']}
    
    # Limpiar el DataFrame de entrada
    df_pred = df_pred.drop(columns=['Sku_encoded', 'UomCode_N_encoded'])

    # Concatenar las predicciones al DataFrame
    df_pred = pd.concat([df_pred, pd.DataFrame(predictions)], axis=1).round(1)
    
    return df_pred

In [219]:
# Obtener los pares únicos de Sku y UomCode_N para cada grupo
df_group_1_unique = df_group_1[['Sku', 'UomCode_N']].drop_duplicates()
df_group_2_unique = df_group_2[['Sku', 'UomCode_N']].drop_duplicates()

In [220]:
# Extraer listas de SKUs y UomCodes correctamente alineadas
lista1_skus = list(df_group_1_unique["Sku"])
lista1_uomcodes = list(df_group_1_unique["UomCode_N"])
lista2_skus = list(df_group_2_unique["Sku"])
lista2_uomcodes = list(df_group_2_unique["UomCode_N"])

In [221]:
# Validar que los SKUs y los UomCodes estén en el encoder
skus_validos1 = [sku for sku in lista1_skus if sku in le.classes_]
uomcodes_validos1 = [uomcode for uomcode in lista1_uomcodes if uomcode in le_uomcode.classes_]

skus_validos2 = [sku for sku in lista2_skus if sku in le.classes_]
uomcodes_validos2 = [uomcode for uomcode in lista2_uomcodes if uomcode in le_uomcode.classes_]

In [222]:
# Predecir para Grupo 1
if skus_validos1 and uomcodes_validos1:
    predicciones_grupo_1 = predict_skus(skus_validos1, uomcodes_validos1, rf_models_1, le, le_uomcode).round(1)

# Predecir para Grupo 2
if skus_validos2 and uomcodes_validos2:
    predicciones_grupo_2 = predict_skus(skus_validos2, uomcodes_validos2, rf_models_2, le, le_uomcode).round(1)

## Armado Bases Finales

In [223]:
p1 = predicciones_grupo_1[["Sku", "UomCode_N", "Weight", "Height", "Length", "Width"]]
p2 = predicciones_grupo_2[["Sku", "UomCode_N", "Weight", "Height", "Length", "Width"]]
pp = predict_pocos[["Sku", "UomCode_N", "Weight", "Height", "Length", "Width"]]
final = pd.concat([pp, p1, p2], ignore_index=True).drop_duplicates()

In [224]:
# Tabla con parejas unicas sku-uomcode
base_uoms = df_cleaned[["Sku", "UomCode_N", "UomCode", "UomQuantity"]].drop_duplicates()

# Unir con tabla de predicciones para tener el uomcode original
final = pd.merge(final, base_uoms, on=["Sku", "UomCode_N"], how="left")

### Base Dims EJD

In [225]:
df_supplier = df_cleaned[["Sku", "UomCode_N", "BaseUnit", "EJD_Weight", "EJD_Height", "EJD_Length", "EJD_Width"]].drop_duplicates()
df_supplier = df_supplier.groupby(["Sku", "UomCode_N", "BaseUnit"])[["EJD_Weight", "EJD_Height", "EJD_Length", "EJD_Width"]].max().reset_index()
df_supplier["BaseUnit"] = df_supplier["BaseUnit"].astype(str).str.lower().map({"true": True, "false": False})
df_supplier = df_supplier.sort_values(by="BaseUnit", ascending=False).drop_duplicates(subset=["Sku", "UomCode_N"])

### Limpieza columna BaseUnit

In [226]:
# Si UomQty es 1 se toma como BaseUnit ese sku

# Base predicciones
index_bu_null = df_supplier[df_supplier["BaseUnit"].isna()].index
df_supplier.loc[index_bu_null, 'BaseUnit'] = True

# Base historico
index_bu_null = df_cleaned[df_cleaned["BaseUnit"].isna()].index
df_cleaned.loc[index_bu_null, 'BaseUnit'] = True

df_supplier.loc[df_supplier[df_supplier['BaseUnit'] == True].index, 'BaseUnit'] = True

In [227]:
# Limpieza base cleaned
true_clean_i = df_cleaned[df_cleaned['BaseUnit'] == 'True'].index
df_cleaned.loc[true_clean_i, 'BaseUnit'] = True

false_clean_i = df_cleaned[df_cleaned['BaseUnit'] == 'False'].index
df_cleaned.loc[false_clean_i, 'BaseUnit'] = False

In [228]:
df_supplier_na_base = df_supplier[df_supplier["BaseUnit"].isna()]
df_cleaned_na_base = df_cleaned[df_cleaned["BaseUnit"] == 0]

### Union con base Final

In [229]:
final_w_supplier = pd.merge(final, df_supplier, on=["Sku", "UomCode_N"], how="left")

### Aplicar Threshold a Predicciones

Aplicar a Skus que son unidades base unicamente

In [230]:
final_w_supplierT = final_w_supplier[final_w_supplier['BaseUnit'] == True]
final_w_supplierF = final_w_supplier[final_w_supplier['BaseUnit'] == False]

df_cleanedT = df_cleaned[df_cleaned['BaseUnit'] == True]
df_cleanedF = df_cleaned[df_cleaned['BaseUnit'] == False]

In [231]:
#### PREDICCIONES
# Threshold para skus con peso menor a 1
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_weight"] = final_w_supplierT["EJD_Weight"] + 0.31
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_height"] = final_w_supplierT["EJD_Height"] + 3
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_length"] = final_w_supplierT["EJD_Length"] + 3
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_width"] = final_w_supplierT["EJD_Width"] + 3

# Threshold para skus con peso mayor a 1
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] >= 1, "th_weight"] = final_w_supplierT["EJD_Weight"] + 3
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] >= 1, "th_height"] = final_w_supplierT["EJD_Height"] + 5
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] >= 1, "th_length"] = final_w_supplierT["EJD_Length"] + 5
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] >= 1, "th_width"] = final_w_supplierT["EJD_Width"] + 5

#### HISTORICO
# Threshold para skus con peso menor a 1
df_cleanedT.loc[df_cleanedT["EJD_Weight"] < 1, "th_weight"] = df_cleanedT["EJD_Weight"] + 0.31
df_cleanedT.loc[df_cleanedT["EJD_Weight"] < 1, "th_height"] = df_cleanedT["EJD_Height"] + 3
df_cleanedT.loc[df_cleanedT["EJD_Weight"] < 1, "th_length"] = df_cleanedT["EJD_Length"] + 3
df_cleanedT.loc[df_cleanedT["EJD_Weight"] < 1, "th_width"] = df_cleanedT["EJD_Width"] + 3

# Threshold para skus con peso mayor a 1
df_cleanedT.loc[df_cleanedT["EJD_Weight"] >= 1, "th_weight"] = df_cleanedT["EJD_Weight"] + 3
df_cleanedT.loc[df_cleanedT["EJD_Weight"] >= 1, "th_height"] = df_cleanedT["EJD_Height"] + 5
df_cleanedT.loc[df_cleanedT["EJD_Weight"] >= 1, "th_length"] = df_cleanedT["EJD_Length"] + 5
df_cleanedT.loc[df_cleanedT["EJD_Weight"] >= 1, "th_width"] = df_cleanedT["EJD_Width"] + 5

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\778330037.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_weight"] = final_w_supplierT["EJD_Weight"] + 0.31
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\778330037.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] < 1, "th_height"] = final_w_supplierT["EJD_Height"] + 3
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\778330

In [232]:
# Pesos y dimensiones finales (aplicando el threshold)

#### Sobrepasando limite superior del rango ########
final_w_supplierT.loc[final_w_supplierT["Weight"] > final_w_supplierT["th_weight"], "fail_weight"] = 1
final_w_supplierT.loc[final_w_supplierT["Weight"] > final_w_supplierT["th_weight"], "Final_Weight"] = final_w_supplierT["th_weight"]
final_w_supplierT.loc[(final_w_supplierT["Weight"] <= final_w_supplierT["th_weight"]) & 
(final_w_supplierT["Weight"] >= final_w_supplierT["EJD_Weight"]), "Final_Weight"] = final_w_supplierT["Weight"]

final_w_supplierT.loc[final_w_supplierT["Height"] > final_w_supplierT["th_height"], "fail_eight"] = 1
final_w_supplierT.loc[final_w_supplierT["Height"] > final_w_supplierT["th_height"], "Final_Height"] = final_w_supplierT["th_height"]
final_w_supplierT.loc[(final_w_supplierT["Height"] <= final_w_supplierT["th_height"]) &
(final_w_supplierT["Height"] >= final_w_supplierT["EJD_Height"]), "Final_Height"] = final_w_supplierT["Height"]

final_w_supplierT.loc[final_w_supplierT["Length"] > final_w_supplierT["th_length"], "fail_length"] = 1
final_w_supplierT.loc[final_w_supplierT["Length"] > final_w_supplierT["th_length"], "Final_Length"] = final_w_supplierT["th_length"]
final_w_supplierT.loc[(final_w_supplierT["Length"] <= final_w_supplierT["th_length"]) & 
(final_w_supplierT["Length"] >= final_w_supplierT["EJD_Length"]), "Final_Length"] = final_w_supplierT["Length"]

final_w_supplierT.loc[final_w_supplierT["Width"] > final_w_supplierT["th_width"], "fail_width"] = 1
final_w_supplierT.loc[final_w_supplierT["Width"] > final_w_supplierT["th_width"], "Final_Width"] = final_w_supplierT["th_width"]
final_w_supplierT.loc[(final_w_supplierT["Width"] <= final_w_supplierT["th_width"]) & 
(final_w_supplierT["Width"] >= final_w_supplierT["EJD_Width"]), "Final_Width"] = final_w_supplierT["Width"]


#### Sobrepasando limite inferior del rango ########
final_w_supplierT.loc[final_w_supplierT["Weight"] < final_w_supplierT["EJD_Weight"], "fail_weight"] = 1
final_w_supplierT.loc[final_w_supplierT["Weight"] < final_w_supplierT["EJD_Weight"], "Final_Weight"] = final_w_supplierT["EJD_Weight"]

final_w_supplierT.loc[final_w_supplierT["Height"] < final_w_supplierT["EJD_Height"], "fail_eight"] = 1
final_w_supplierT.loc[final_w_supplierT["Height"] < final_w_supplierT["EJD_Height"], "Final_Height"] = final_w_supplierT["EJD_Height"]

final_w_supplierT.loc[final_w_supplierT["Length"] < final_w_supplierT["EJD_Length"], "fail_length"] = 1
final_w_supplierT.loc[final_w_supplierT["Length"] < final_w_supplierT["EJD_Length"], "Final_Length"] = final_w_supplierT["EJD_Length"]

final_w_supplierT.loc[final_w_supplierT["Width"] < final_w_supplierT["EJD_Width"], "fail_width"] = 1
final_w_supplierT.loc[final_w_supplierT["Width"] < final_w_supplierT["EJD_Width"], "Final_Width"] = final_w_supplierT["EJD_Width"]


C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\737832289.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierT.loc[final_w_supplierT["Weight"] > final_w_supplierT["th_weight"], "fail_weight"] = 1
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\737832289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierT.loc[final_w_supplierT["Weight"] > final_w_supplierT["th_weight"], "Final_Weight"] = final_w_supplierT["th_weight"]
C:\Users\ecalderon\AppData\Local\Temp\ipykerne

In [233]:
# Pesos y dimensiones finales (aplicando el threshold)

#### Skus que no tienen dimensiones en EJD ########

final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "fail_weight"] = np.nan
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "Final_Weight"] = final_w_supplierT["Weight"]
final_w_supplierF["Final_Weight"] = final_w_supplierF["Weight"]

final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "fail_eight"] = np.nan
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "Final_Height"] = final_w_supplierT["Height"]
final_w_supplierF["Final_Height"] = final_w_supplierF["Height"]

final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "fail_length"] = np.nan
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "Final_Length"] = final_w_supplierT["Length"]
final_w_supplierF["Final_Length"] = final_w_supplierF["Length"]

final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "fail_width"] = np.nan
final_w_supplierT.loc[final_w_supplierT["EJD_Weight"] == 0, "Final_Width"] = final_w_supplierT["Width"]
final_w_supplierF["Final_Width"] = final_w_supplierF["Width"]

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\160907871.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierF["Final_Weight"] = final_w_supplierF["Weight"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\160907871.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierF["Final_Height"] = final_w_supplierF["Height"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\160907871.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [234]:
# Quitando dimensiones a Skus que no son unidad base

final_w_supplierF[['EJD_Weight', 'EJD_Height', 'EJD_Length', 'EJD_Width']] = np.nan

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\1779026084.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_w_supplierF[['EJD_Weight', 'EJD_Height', 'EJD_Length', 'EJD_Width']] = np.nan


In [235]:
final_w_supplier = pd.concat([final_w_supplierT, final_w_supplierF], ignore_index=True)

### Metricas de Desviacion con la Especificacion

In [236]:
# SUMAR DIMENSIONES
df_cleanedT["Sum_Dim"] = df_cleanedT["Height"] + df_cleanedT["Length"] + df_cleanedT["Width"]
df_cleanedF["Sum_Dim"] = df_cleanedF["Height"] + df_cleanedF["Length"] + df_cleanedF["Width"]
df_cleanedT["Sum_Dim_EJD_Sup"] = df_cleanedT["th_height"] + df_cleanedT["th_length"] + df_cleanedT["th_width"]
df_cleanedT["Sum_Dim_EJD_Low"] = df_cleanedT["EJD_Height"] + df_cleanedT["EJD_Length"] + df_cleanedT["EJD_Width"]

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\172691531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleanedT["Sum_Dim"] = df_cleanedT["Height"] + df_cleanedT["Length"] + df_cleanedT["Width"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\172691531.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleanedF["Sum_Dim"] = df_cleanedF["Height"] + df_cleanedF["Length"] + df_cleanedF["Width"]
C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\172691531.py:4: SettingWithCopyWarning: 


In [237]:
# MEDIANA PESO Y DIMS
df_metricaT = df_cleanedT.groupby(["Sku", 'UomCode_N', 'BaseUnit'])[["Weight", "Sum_Dim", "EJD_Weight", "th_weight", "Sum_Dim_EJD_Low", "Sum_Dim_EJD_Sup"]].median().round(2).reset_index()
df_metricaF = df_cleanedF.groupby(["Sku", 'UomCode_N', 'BaseUnit'])[['Weight', "Sum_Dim"]].median().round(2).reset_index()

In [238]:
# FLAG PESO
i_w_low = df_metricaT[(df_metricaT["Weight"] < df_metricaT["EJD_Weight"])].index
i_w_high = df_metricaT[(df_metricaT["Weight"] > df_metricaT["th_weight"])].index

df_metricaT["Weight_Flag"] = "Within range"
df_metricaT.loc[i_w_low, "Weight_Flag"] = "Underweight"
df_metricaT.loc[i_w_high, "Weight_Flag"] = "Overweight"

df_metricaF["Weight_Flag"] = "No EJD Dims"

In [239]:
# FLAG DIMS
i_w_lowD = df_metricaT[(df_metricaT["Sum_Dim"] < df_metricaT["Sum_Dim_EJD_Low"])].index
i_w_highD = df_metricaT[(df_metricaT["Sum_Dim"] > df_metricaT["Sum_Dim_EJD_Sup"])].index

df_metricaT["DIM_Flag"] = "Within range"
df_metricaT.loc[i_w_lowD, "DIM_Flag"] = "Under"
df_metricaT.loc[i_w_highD, "DIM_Flag"] = "Over"

df_metricaF["DIM_Flag"] = "No EJD Dims"

In [240]:
df_cleaned = pd.concat([df_cleanedT, df_cleanedF], ignore_index=True)
df_metrica = pd.concat([df_metricaT, df_metricaF], ignore_index=True)

# SALIDA EVP PARA SUBIR POR MEDIO DEL UOMCODE

In [241]:
final_w_supplier_evp = final_w_supplier[['Sku', 'UomCode', 'UomQuantity', 'Final_Weight', 'Final_Height', 'Final_Length', 'Final_Width']]

In [242]:
from datetime import datetime
fecha = datetime.today().strftime('%Y%m%d')

# Suponiendo que tu DataFrame se llama df
chunk_size = 1000  # Tamaño de cada archivo
chunks = np.array_split(final_w_supplier_evp, len(final_w_supplier_evp) // chunk_size + (1 if len(final_w_supplier_evp) % chunk_size != 0 else 0))  # Dividir en partes

# Guardar cada chunk en un CSV
list(map(lambda x: x[1].to_csv(f"{fecha}_df_evp_{x[0]+1}.csv", index=False), enumerate(chunks)))

print(f"Se han creado {len(chunks)} archivos CSV.")

C:\Users\ecalderon\AppData\Local\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Se han creado 17 archivos CSV.


# Ventas

In [243]:
df_ventas = df_cleaned[['TranDate', 'InvDate', 'trackingnum','Sku', 'UomCode_N', 'UomQuantity', 'SalesStatusEVP','TotalSales', 'TotalCost']].drop_duplicates()

In [244]:
# crear campos de temporalidad

df_ventas["Year"] = df_ventas["TranDate"].dt.year
df_ventas["Month"] = df_ventas["TranDate"].dt.month_name()
df_ventas["YearMonth"] = df_ventas["TranDate"].dt.strftime("%Y-%m")

In [245]:
# Agrupar por Año-Mes y SKU, sumando las ventas
ventas_por_mes = df_ventas.groupby(["YearMonth", "Sku", "UomCode_N"])["TotalSales"].sum().reset_index()
ventas_por_mes = ventas_por_mes[ventas_por_mes['TotalSales'] != 0.00]

In [246]:
# Colocar los meses en columnas

df_transposed = ventas_por_mes.pivot_table(index=["Sku", 'UomCode_N'], columns="YearMonth", values="TotalSales", aggfunc="sum").round(2).reset_index()

In [247]:
# Calcular tendencia

# Columnas de meses
ventas_cols = df_transposed.columns[2:]

# Calcular la variación promedio mes a mes
df_transposed["Tendencia"] = df_transposed[ventas_cols].pct_change(axis=1).mean(axis=1, skipna=True).round(2)

# Reemplazar NaN por 0 si no hubo ventas en ningún mes
df_transposed = df_transposed.fillna(0)

C:\Users\ecalderon\AppData\Local\Temp\ipykernel_5568\3470377078.py:7: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_transposed["Tendencia"] = df_transposed[ventas_cols].pct_change(axis=1).mean(axis=1, skipna=True).round(2)


In [277]:
df_metrica.isnull().sum()

Sku                    0
UomCode_N              0
BaseUnit               0
Weight                 0
Sum_Dim                0
EJD_Weight          2887
th_weight           2887
Sum_Dim_EJD_Low     2887
Sum_Dim_EJD_Sup     2887
Weight_Flag            0
DIM_Flag               0
2025-03            14484
2025-04            14484
Tendencia          14484
dtype: int64

### Unir al dataframe de Metricas

In [248]:
df_metrica = df_metrica.merge(df_transposed, on=["Sku", "UomCode_N"], how="left")

## CSVs

In [249]:
final_w_supplier.to_csv("df_predicciones.csv", index=False)
df_sorted.to_csv("df_historico.csv", index=False)
df_metrica.to_csv("df_metrica.csv", index=False)

## Query de Salida

### Por tracking

In [250]:
server = '10.12.200.59' #ip del DB es 10.11.100.103
database = 'Maxwarehouse'

# Crear el engine usando la cadena de conexión para SQL Server con pyodbc
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Ejecutar consulta 

query2 = """

WITH BOXFECHAS AS (
	Select TRACKING_NUMBER,OrderDate, case when len(DIMENSIONS)>=5 then CAST(SUBSTRING(DIMENSIONS, 1, CHARINDEX('x', DIMENSIONS) - 1) AS float) else 0.00 end DimHeightBOX,
			case when len(DIMENSIONS)>=5 then CAST(SUBSTRING(DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1, 
								CHARINDEX('x', DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1) - CHARINDEX('x', DIMENSIONS) - 1) AS float)  else 0.00 end DimLengthBOX, 
			case when len(DIMENSIONS)>=5 then CAST(RIGHT(DIMENSIONS, LEN(DIMENSIONS) - CHARINDEX('x', DIMENSIONS, CHARINDEX('x', DIMENSIONS) + 1)) AS float)  else 0.00 end  DimWidthBOX, 
			CCN_ORDER_NUMBER
			FROM 
				BOX 
			WHERE ISDATE(OrderDate) = 1),

endicia_semana_mes AS (
		SELECT DISTINCT  
		[Postmark_Date] AS TranDate, 
		[Transaction_Date (GMT)] AS InvDate, 
		REPLACE([Tracking Number], '''', '') AS trackingnum, 
		[Weight_(lb)], 
		[Package Height], 
		[Package Length], 
		[Package Width],
		[Transaction_Amount] as DiscountedNetCharge
	FROM Endiciatransacprovision
	WHERE CAST([Transaction_Date (GMT)] AS date) >= DATEADD(MONTH, -4, GETDATE())
	UNION ALL
	SELECT DISTINCT  
		[Postmark Date] AS TranDate, 
		[Transaction Date (GMT)] AS InvDate, 
		REPLACE([Tracking Number], '''', '') AS trackingnum, 
		[Weight (lb)], 
		[Package Height], 
		[Package Length], 
		[Package Width],
		[Transaction Amount] as DiscountedNetCharge
	FROM endicia_semanal
	WHERE CAST([Transaction Date (GMT)] AS date) >= DATEADD(MONTH, -2, GETDATE())
),

Factura3meses AS (
	SELECT DISTINCT
        cast(InvoiceDate as date) TranDate, cast(InvoiceDate as date) InvDate, GroundTrackingIDPrefix+ExpressorGroundTrackingID trackingnum,
        MAX(ActualWeightAmount) AS max_weight, 
		CASE WHEN DimHeight !='' THEN cast(DimHeight as float) ELSE  DimHeightBOX END DimHeight,
		CASE WHEN DimLength !='' THEN cast(DimLength as float) ELSE  DimLengthBOX END DimLength, 
		CASE WHEN DimWidth !='' THEN cast(DimWidth as float) ELSE  DimWidthBOX END DimWidth,
		max(NetChargeAmount) as DiscountedNetCharge
    FROM 
        FEDEXHISTORICO
	LEFT JOIN 
			BOXFECHAS ON GroundTrackingIDPrefix+ExpressorGroundTrackingID = TRACKING_NUMBER
    WHERE 
        cast(InvoiceDate as date) >= DATEADD(MONTH, -3, GETDATE()) 
    GROUP BY 
        cast(InvoiceDate as date), cast(InvoiceDate as date), GroundTrackingIDPrefix+ExpressorGroundTrackingID,
		CASE WHEN DimHeight !='' THEN cast(DimHeight as float) ELSE  DimHeightBOX END ,
		CASE WHEN DimLength !='' THEN cast(DimLength as float) ELSE  DimLengthBOX END , 
		CASE WHEN DimWidth !='' THEN cast(DimWidth as float) ELSE  DimWidthBOX END 
    UNION ALL
    SELECT 
        cast(TransactionDate as date) TranDate, cast(InvoiceDate as date) InvDate,TrackingNumber,
        max(case when BilledWeight != 0 then BilledWeight else EnteredWeight end) as max_weight,
		CASE WHEN PackageDimensions !='' THEN 
			CAST(SUBSTRING(PackageDimensions, 1, CHARINDEX('x', PackageDimensions) - 1) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(SUBSTRING(DetailKeyedDim, 1, CHARINDEX('x', DetailKeyedDim) - 1) AS float) else DimHeightBOX end END AS Valor1,

		CASE WHEN PackageDimensions !='' THEN 
			CAST(SUBSTRING(PackageDimensions, CHARINDEX('x', PackageDimensions) + 1, 
                        CHARINDEX('x', PackageDimensions, CHARINDEX('x', PackageDimensions) + 1) - CHARINDEX('x', PackageDimensions) - 1) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(SUBSTRING(DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1, 
                        CHARINDEX('x', DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1) - CHARINDEX('x', DetailKeyedDim) - 1) AS float)  else DimLengthBOX end END  AS Valor2,

		CASE WHEN PackageDimensions !='' THEN 
			CAST(RIGHT(PackageDimensions, LEN(PackageDimensions) - CHARINDEX('x', PackageDimensions, CHARINDEX('x', PackageDimensions) + 1)) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(RIGHT(DetailKeyedDim, LEN(DetailKeyedDim) - CHARINDEX('x', DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1)) AS float)  else DimWidthBOX end END  AS Valor3,
			sum(netamount) as DiscountedNetCharge
    FROM 
        UPSNEWFORMAT
	LEFT JOIN 
			BOXFECHAS ON TrackingNumber = TRACKING_NUMBER
    WHERE 
        cast(InvoiceDate as date) >= DATEADD(MONTH, -3, GETDATE())
    GROUP BY 
        cast(TransactionDate as date), cast(InvoiceDate as date), TrackingNumber,
		CASE WHEN PackageDimensions !='' THEN 
			CAST(SUBSTRING(PackageDimensions, 1, CHARINDEX('x', PackageDimensions) - 1) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(SUBSTRING(DetailKeyedDim, 1, CHARINDEX('x', DetailKeyedDim) - 1) AS float) else DimHeightBOX end END ,

		CASE WHEN PackageDimensions !='' THEN 
			CAST(SUBSTRING(PackageDimensions, CHARINDEX('x', PackageDimensions) + 1, 
                        CHARINDEX('x', PackageDimensions, CHARINDEX('x', PackageDimensions) + 1) - CHARINDEX('x', PackageDimensions) - 1) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(SUBSTRING(DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1, 
                        CHARINDEX('x', DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1) - CHARINDEX('x', DetailKeyedDim) - 1) AS float)  else DimLengthBOX end END ,

		CASE WHEN PackageDimensions !='' THEN 
			CAST(RIGHT(PackageDimensions, LEN(PackageDimensions) - CHARINDEX('x', PackageDimensions, CHARINDEX('x', PackageDimensions) + 1)) AS float)
		ELSE
			case when len(DetailKeyedDim)>=5 then CAST(RIGHT(DetailKeyedDim, LEN(DetailKeyedDim) - CHARINDEX('x', DetailKeyedDim, CHARINDEX('x', DetailKeyedDim) + 1)) AS float)  else DimWidthBOX end END 

	UNION ALL
	SELECT DISTINCT  TranDate, InvDate, trackingnum, 
	[Weight_(lb)], case when [Package Height] is not null then [Package Height] else DimHeightBOX END , 
	case when [Package Length] is not null then [Package Length] else DimLengthBOX END, 
	case when [Package Width] is not null then [Package Width] else DimWidthBOX END , DiscountedNetCharge 
	FROM 
		endicia_semana_mes
	LEFT JOIN BOXFECHAS ON TRACKING_NUMBER = trackingnum
	WHERE trackingnum IS NOT NULL
	and cast(InvDate as date) >= DATEADD(MONTH, -4, GETDATE()) --se colocan 4 meses porque las dims de USPS vienen mensualmente
),

DWVentasUnicas as (
	Select 
		SalesOrderNumber, Sku
	from DWShippingInfo as dws
	where cast(SalesOrderDate as date ) between dateadd(MONTH,-4, cast(GETDATE() as date)) and cast(GETDATE() as date)
	group by SalesOrderNumber, Sku
),

CantidadSkusxVenta as (
	select
		SalesOrderNumber, count(Sku) CantidadSkus
	from DWVentasUnicas dw 
	group by SalesOrderNumber
	having count(Sku) =1
),

PesosfacturaVentas as (
	select  
		TranDate, InvDate, dw.trackingnum, mw.FulfillmentOrderNumber, dw.DiscountedNetCharge, max_weight, DimHeight, DimLength, 
		DimWidth, mw.Sku, mw.UomCode, mw.UomQuantity, mw.EstimatedShippingCost
	FROM Factura3meses dw
	INNER JOIN DWShippingInfo  mw
		ON dw.trackingnum = mw.trackingnum
	WHERE dw.trackingnum !=''
	and SalesOrderNumber in (select SalesOrderNumber from CantidadSkusxVenta)
	and cast(UomQuantity as float)/cast(Quantity as float)= 1
)

select InvDate,trackingnum, FulfillmentOrderNumber, sum(DiscountedNetCharge) as DiscountedNetCharge, Sku, UomCode, UomQuantity, 
max(EstimatedShippingCost) as EstimatedCost  
from PesosfacturaVentas
group by InvDate,trackingnum, FulfillmentOrderNumber, Sku, UomCode, UomQuantity
having InvDate >= DATEADD(WEEK, -2, GETDATE())

"""

campos_salida = pd.read_sql(query2, con=engine)

In [251]:
campos_salida = normalize_uom_code(campos_salida)

In [252]:
base_salida = final_w_supplier[["Sku", "UomCode_N", "Final_Height", "Final_Length", "Final_Width", "Final_Weight"]]

In [253]:
salida_evp = pd.merge(campos_salida, base_salida, on=["Sku", "UomCode_N"], how="left")

In [254]:
salida_evp = salida_evp[['trackingnum', 'FulfillmentOrderNumber', "Final_Height", "Final_Length", "Final_Width", "Final_Weight", 'DiscountedNetCharge']]
salida_evp.rename(columns={"trackingnum": "TrackingNumber", 
                           "FulfillmentOrderNumber": "ShipperReference",
                           "Final_Height": "PackageHeight",
                           "Final_Length": "PackageLength",
                           "Final_Width": "PackageWidth",
                           "Final_Weight": "BilledWeight"}, inplace=True)

In [255]:
salida_evp.to_csv("df_evp.csv", index=False)

## Comparacion estimated vs real cost

In [256]:
campos_salida.rename(columns={'DiscountedNetCharge': 'Cost'}, inplace=True)

In [257]:
# convertir tipo de dato
campos_salida['Cost'] = pd.to_numeric(campos_salida['Cost'], errors='coerce')
campos_salida['EstimatedCost'] = pd.to_numeric(campos_salida['EstimatedCost'], errors='coerce')

In [258]:
# Variacion porcentual del costo
campos_salida['variacion_costo'] = ((campos_salida['Cost'] - campos_salida['EstimatedCost']) / campos_salida['EstimatedCost']).round(2)

In [259]:
# Semana del reporte
campos_salida['Semana'] = campos_salida['InvDate'].dt.isocalendar().week

In [260]:
campos_salida[(campos_salida['variacion_costo'] < 0) & (campos_salida['EstimatedCost'] != 0)].sort_values('variacion_costo', ascending=True)

,InvDate,trackingnum,FulfillmentOrderNumber,Cost,Sku,UomCode,UomQuantity,EstimatedCost,UomCode_N,variacion_costo,Semana
4195,2025-04-05,1Z53E8A6YW00750628,PO-259166025-EJD,-27.06,ACE-4259750,EA,1.0,8.03,EA,-4.37,14
2904,2025-04-05,1Z53E8A60300752080,PO-259168450-EJD,-31.42,ACE-4300901,EA,1.0,10.20,EA,-4.08,14
2887,2025-04-05,1Z53E8A60300750831,PO-259164985-EJD,-31.42,ACE-8314668,PACK_3,3.0,10.73,PK_3,-3.93,14
7354,2025-04-05,1Z53E9290350398416,PO-259168067-EJD,-27.06,JEN-4282-1017,EA,1.0,9.37,EA,-3.89,14
2900,2025-04-05,1Z53E8A60300751616,PO-259164982-EJD,-27.06,ACE-4595385,PACK_12,12.0,9.37,PK_12,-3.89,14
...,...,...,...,...,...,...,...,...,...,...,...
3040,2025-04-05,1Z53E8A60300759074,PO-259179231-EJD,9.26,ACE-12808,EA,1.0,9.37,EA,-0.01,14
4061,2025-04-05,1Z53E8A60350750390,PO-259190495-EJD,9.26,ACE-7284011,EA,1.0,9.37,EA,-0.01,14
7258,2025-04-05,1Z53E9290300396379,PO-259199295-EJD,9.52,ACE-44728,EA,1.0,9.60,EA,-0.01,14
9842,2025-04-05,1Z53FA37YW21461088,SO-259581586-A,9.25,EJD-9098828-A,EA,1.0,9.31,EA,-0.01,14


In [274]:
campos_salida.to_csv('costos.csv', mode='a', header=False, index=False)

## Cuantificación Impacto

In [262]:
server = ''
database = ''

# Crear el engine usando la cadena de conexión para SQL Server con pyodbc
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server")

# Ejecutar consulta 

query = """
select * from [stg-MaxWarehouse].[EvpCsv].[EVPAlllSKUsDimNWeight]
"""

df_evp = pd.read_sql(query, con=engine)

### Skus que cambiaron sus dimensiones

In [263]:
df_pred = final_w_supplier[['Sku', 'UomCode_N', 'UomCode', 'UomQuantity', 'BaseUnit', 'Final_Weight', 'Final_Height', 'Final_Length', 'Final_Width']]
df_evp = df_evp[['EVP SKU', 'EVP UOM CODE', 'EVP UOM QTY', 'EVP Weight', 'EVP Height', 'EVP Length', 'EVP Width']]

In [264]:
df_merge = df_pred.merge(
    df_evp,  
    left_on=['Sku', 'UomCode'], 
    right_on=['EVP SKU', 'EVP UOM CODE'], 
    how='left'
)

In [265]:
# EVP solo toma sku base unit
df_bu = df_merge[df_merge['BaseUnit'] == True]

# Quitando nulos
df_bu = df_bu[~df_bu['EVP Width'].isnull()]

In [266]:
# convertir tipo de dato
df_bu['EVP Weight'] = pd.to_numeric(df_bu['EVP Weight'], errors='coerce')
df_bu['EVP Height'] = pd.to_numeric(df_bu['EVP Height'], errors='coerce')
df_bu['EVP Length'] = pd.to_numeric(df_bu['EVP Length'], errors='coerce')
df_bu['EVP Width'] = pd.to_numeric(df_bu['EVP Width'], errors='coerce')

In [267]:
# Fecha del reporte
df_bu['Fecha'] = pd.to_datetime('today').normalize()
df_bu['Semana'] = df_bu['Fecha'].dt.isocalendar().week

In [268]:
# Suma dimensiones
df_bu['Sum_Dims_Pred'] = df_bu['Final_Height'] + df_bu['Final_Length'] + df_bu['Final_Width']
df_bu['Sum_Dims_EVP'] = df_bu['EVP Height'] + df_bu['EVP Length'] + df_bu['EVP Width']

In [269]:
# Variacion porcentual
df_bu['variacion_weight'] = ((df_bu['Final_Weight'] - df_bu['EVP Weight']) / df_bu['EVP Weight']).round(2)
df_bu['variacion_dims'] = ((df_bu['Sum_Dims_Pred'] - df_bu['Sum_Dims_EVP']) / df_bu['Sum_Dims_EVP']).round(2)

In [270]:
df_bu = df_bu.rename(columns={'EVP Weight': 'EVP_Weight', 'EVP Height': 'EVP_Height', 'EVP Length': 'EVP_Length', 'EVP Width': 'EVP_Width'})

In [275]:
df_bu.to_csv('cuantificacion_skus.csv', mode='a', header=False, index=False)

In [272]:
def send_email_final():
    # Configuración del correo
    
    # Contenido del correo
    subject = "🔔 Alerta: Generación de predicciones de pesos completada"
    body = "El modelo de predicción ha finalizado correctamente. Sube los resultados a EVP."

    # Crear el mensaje
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = ", ".join(recipient_list)

    try:
        # Conectar al servidor SMTP (ejemplo para Gmail)
        server = smtplib.SMTP("smtp.office365.com", 587)  # Usa "smtp.office365.com" para Outlook
        server.starttls()  # Seguridad TLS
        server.login(sender_email, sender_password)  # Autenticación
        server.sendmail(sender_email, recipient_list, msg.as_string())  # Enviar
        server.quit()
        print("✅ Correo enviado correctamente")
    except Exception as e:
        print(f"⚠ Error al enviar el correo: {e}")

# Llamar a la función al final del proceso
send_email_final()

✅ Correo enviado correctamente


In [273]:
import nbformat

# Cargar el notebook
with open("Scrip_Pesos_y_Dims.ipynb", "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)

# Extraer el código de cada celda y guardarlo en un archivo .py
codigo = "\n".join(cell["source"] for cell in notebook["cells"] if cell["cell_type"] == "code")

# Guardar como .py
with open("Scrip_Pesos_y_Dims.py", "w", encoding="utf-8") as f:
    f.write(codigo)